In [ ]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.cross_validation
import joblib
from time import time
import logging

import fs_loader
import model
import plotting

import GPy
import GPy.kern as K

%matplotlib inline


logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

def filter_valid_dates(df, request):
    assert 'longitude' in df.columns and 'latitude' in df.columns
    spatial_df = df[(df.valid_date >= request['start']) & (df.valid_date <= request['end'])].copy()
    assert len(spatial_df) > 0, "date range [%s - %s] not included in data" % (request['start'], request['end'])
    print("Selected %d valid date(s)." % len(spatial_df.valid_date.unique()))
    return spatial_df
    

def calculate_error(df, request):
    predictant = request['predictant']
    model = request['model_name']
    obs_col = predictant + '_observed'

    main_predictor = predictant + '_' + model

    error = df[obs_col] - df[main_predictor]
    return error


def load_dataset(request):
    print("Loading dataset..")
    dataset = fs_loader.load_dataset(request)
    dataset[request['predictant'] + '_error'] = calculate_error(dataset, request)
    dataset = filter_valid_dates(dataset, request)
    print("Done loading dataset.")
    print("Stations in set: %d" % (len(dataset.station_id.unique())))
    return dataset


def split_dataset(dataset, feature_cols, obs_col):
    station_ids = dataset.station_id.unique()
    train_stations, test_stations = sklearn.cross_validation.train_test_split(
        station_ids, test_size=0.10
    )

    X_train = dataset.loc[
        dataset.station_id.isin(train_stations), 
        feature_cols
    ].values

    X_test = dataset.loc[
        dataset.station_id.isin(test_stations),
        feature_cols
    ].values

    y_train = dataset.loc[
        dataset.station_id.isin(train_stations),
        [obs_col]
    ].values

    y_test = dataset.loc[
        dataset.station_id.isin(test_stations),
        [obs_col]
    ].values
    return X_train, X_test, y_train, y_test


def do_verification(kernel, request):
    dataset = load_dataset(request)
    valid_dates = dataset.valid_date.unique()
    grouper = dataset.groupby('valid_date')
    results = []
    for count, (valid_date, df) in enumerate(grouper):
        print("Validating date %s (%d / %d)." % (valid_date, count + 1, len(valid_dates)))
        # TODO Additionally group on forecast hours.
        # TODO Create name generator based on GP objects.
        # TODO Do averaging for multiple cv folds
        mae, rmse = preprocess_and_fit_model(df.copy(), kernel, request)
        results.append([valid_date, mae, rmse])
    result_df = pd.DataFrame(results, columns=['valid_date', 'mae', 'rmse'])
    result_df['name'] = 'name'
    return result_df


def preprocess_and_fit_model(dataset, kernel, request):
    # TODO TdR 17/07/16: preserve original row-mapping for later reference
    # TODO TdR 19/07/16: Try feature embeddings
    observation_column = request['predictant'] + '_error'
    X_train, X_test, y_train, y_test = \
        split_dataset(dataset, request['features'], observation_column)
    gp = fit_model(X_train, y_train, kernel)
    predictions, _ = chunk_predict(gp, X_test)
    mae = sklearn.metrics.mean_absolute_error(y_true=y_test, y_pred=predictions)
    rmse = np.sqrt((sklearn.metrics.mean_squared_error(y_true=y_test, y_pred=predictions)))
    return mae, rmse
    
    
def fit_model(X, y, kernel, *args):
    print("Training GP..")
    start = time()
    gp = GPy.models.GPRegression(X, y, kernel=kernel)
    gp.optimize(messages=True)
    end = time()
    print("Finished GP training (%ds)." % (end - start))
    return gp


def chunk_predict(gp, input_vector, chunk_size=5000):
    preds = []
    var = []
    points_done = 0
    for chunk in chunks(input_vector, chunk_size):
        chunk_preds, chunk_var = gp.predict(chunk)
        preds.append(chunk_preds)
        var.append(chunk_var)
        points_done += len(chunk)
        print("Finished %d / %d predictions.." % (points_done, len(input_vector)))
    return np.concatenate(preds), np.concatenate(var)


def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]


def get_kernel(input_dim):
    kernel = K.Matern52(input_dim, ARD=True) + K.White(input_dim)
    return kernel


def fit_single_model_and_return_dataset(kernel, request):
    dataset = load_dataset(request)
    
    observation_column = request['predictant'] + '_error'
    X_train, X_test, y_train, y_test = \
        split_dataset(dataset, request['features'], observation_column)
    print("Dataset: %d rows, %d features" % X_train.shape)
    gp = fit_model(X_train, y_train, kernel)

    # Validate model
    predictions, _ = chunk_predict(gp, X_test)
    print("MAE: %.3f." % (sklearn.metrics.mean_absolute_error(y_true=y_test, y_pred=predictions)))
    print("RMSE: %.3f." % np.sqrt((sklearn.metrics.mean_squared_error(y_true=y_test, y_pred=predictions))))
    return gp, dataset

In [ ]:
# Spatial prediction
import altitude

def generate_latlon_grid(request):
    top_lat, bot_lat, left_lon, right_lon = request['predict_area']
    res = request['predict_resolution']
    n_lons = int(np.ceil(abs(right_lon - left_lon) / res))
    n_lats = int(np.ceil(abs(top_lat - bot_lat) / res))    
    lat_grid, lon_grid = np.meshgrid(np.linspace(bot_lat, top_lat, n_lats),
                                     np.linspace(left_lon, right_lon, n_lons)
                                    )
    latlon_vector = np.hstack([
            lat_grid.reshape(lat_grid.size, 1), 
            lon_grid.reshape(lon_grid.size, 1)])
    return latlon_vector, n_lats, n_lons


def get_elevations(request):
    e = altitude.ElevationService('.cache/')
    latlon_vector, n_lats, n_lons = generate_latlon_grid(test_request)
    elevations = []
    for point in latlon_vector:
        elevations.append(e.get_elevation(*point))
    return np.array(elevations)


def predict_area(gp, request):
    data, n_lats, n_lons = generate_latlon_grid(request)
    if 'elevation' in request['features']:
        elevations = get_elevations(request)
        data = np.hstack([data, elevations.reshape((elevations.size, 1))])
    print("Predicting for %d points and %d features.." % data.shape)
    y_pred, var = chunk_predict(gp, data)
    y_pred = y_pred.reshape((n_lons, n_lats)).T
    var = var.reshape((n_lons, n_lats)).T
    return y_pred, var

In [ ]:
from IPython.display import display
import datetime as dt
from time import time

# Available features.
#     ['station_id' 'latitude' 'longitude' 'elevation' 'forecast_hour'
#  'valid_date' 'TT2m_observed' 'TT2m_ModelMix' 'FF10m_ModelMix' 'valid_hour'
#  'valid_month']

test_request = {
    'predictant': 'TT2m',
    'model_elements': ['TT2m', 'FF10m'],
    'features': ['latitude', 'longitude', 'elevation'],
    'forecast_hours': [42],
    'model_name': 'ModelMix',
    'start': dt.datetime(2015, 11, 1),
    'end': dt.datetime(2015, 11, 10),
    'predict_area': (50, 45, 5, 10),  # Swiss Alps
#     'predict_area': (60, 33, -12, 20),  # Europe
    'predict_resolution': 0.1
}

start_time = time()
kernel = get_kernel(len(test_request['features']))
results = do_verification(kernel, test_request)
# gp, dataset = fit_single_model_and_return_dataset(kernel, test_request)
# area_prediction, area_std = predict_area(gp, test_request)
# plotting.plot_prediction_distribution(area_prediction)
# plotting.plot_area(dataset, test_request, area_prediction)
end_time = time()
print("Total processing time: %ds" % (end_time - start_time))

In [ ]:
# Plot elevations.
import altitude
import numpy as np
import matplotlib.pyplot as plt

e = altitude.ElevationService('.cache/')
latlon_vector, n_lats, n_lons = generate_latlon_grid(test_request)
elevations = []
for point in latlon_vector:
    elevations.append(e.get_elevation(*point))
elevations = np.array(elevations).reshape((n_lons, n_lats)).T
top_lat, bot_lat, left_lon, right_lon = test_request['predict_area']

plt.contour(
    elevations,
    np.arange(0, 5000, 100),
    extent=(left_lon, right_lon, bot_lat, top_lat),
#     antialiased=True,
    zorder=999
)
plt.colorbar()